In [188]:
from collections import deque, defaultdict
from functools import cache

In [203]:
class Elves(object):
    directions = {
            "N":    (-1,0),
            "NW":   (-1,-1),
            "W":    (0,-1),
            "SW":   (1,-1),
            "S":    (1,0),
            "SE":   (1,1),
            "E":    (0,1),
            "NE":   (-1,1),
        }


    def __init__(self,file) -> None:
        self.views = deque(
            [("N",(self.directions["N"],self.directions["NE"],self.directions["NW"])),
            ("S",(self.directions["S"],self.directions["SE"],self.directions["SW"])),
            ("W",(self.directions["W"],self.directions["NW"],self.directions["SW"])),
            ("E",(self.directions["E"],self.directions["NE"],self.directions["SE"]))]
        )
        self._file = file
        self.load()

    @property
    def extent(self):

        return ((min([x[0] for x in self._elves]), min([x[1] for x in self._elves])),
        (max([x[0] for x in self._elves]), max([x[1] for x in self._elves])))


    @property
    def score(self):
        return (1+ self.extent[1][1] - self.extent[0][1]) * (1+ self.extent[1][0] - self.extent[0][0]) - len(self._elves)
        
    def __repr__(self) -> str:
        repr = ''

        for r in range(self.extent[0][0], self.extent[1][0]+1):
            for c in range(self.extent[0][1], self.extent[1][1]+1):
                if (r,c) in self._elves:
                    repr += '#'
                else:
                    repr += '.'

            repr += '\n'

        return repr

    def load(self):
        with open('./assets/input_day_23.txt', 'r') as file:
            input = file.read().splitlines()

            elves = {}
            for r,line in enumerate(input):
                for c,v in enumerate(line):
                    if v == '#':
                        elves[(r,c)] = None

        self._elves = elves
        

    @cache
    def target(self,pos,vec):
        return tuple([x+y for x,y in zip(pos,vec)])


    def proposition(self,elf):
        
        no_move = True
        first_empty = None
        for view in self.views:
            
            a = [self.target(elf,drct) in self._elves for drct in view[1]]
            if any(a):
                no_move = False
            elif first_empty is None:
                    first_empty = view[0]

        if no_move or first_empty is None:
            return elf
        elif first_empty:
            return self.target(elf,self.directions[first_empty])

    def get_moves(self):
        
        propositions = defaultdict(list)
        for elf in self._elves:
            propositions[self.proposition(elf)].append(elf)
            
        return {k:v[0] for k,v in propositions.items() if len(v) == 1 and not k == v[0]}


    def move(self):

        moves = self.get_moves()

        for dest,origin in moves.items():
            self._elves.pop(origin)
            self._elves[dest] = origin

        self.views.rotate(-1)

        return moves    

In [206]:
pt1 = Elves('./assets/input_day_23.txt')
for i in range(10):
    _ = pt1.move()

print(f"{pt1.score} empty spaces after {i+1} rounds")


4109 empty spaces after 10 rounds


In [207]:
pt2 = Elves('./assets/input_day_23.txt')
mvs = None
i = 0
while mvs != {}:
    mvs = pt2.move()
    i += 1

print(f"no more moves after: {i} rounds")

no more moves after: 1055 rounds
